In [ ]:
# necessary imports
import os, sys
from dotenv import load_dotenv
import tweepy as tweepy
from neo4j import GraphDatabase, basic_auth
load_dotenv()

In [ ]:
# authenticaing twitter api user
#consumer_key = ''
#consumer_secret = ''
#access_token =''
#access_token_secret = ''

#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_token_secret)


bearer_token = os.getenv('BEARER_TOKEN')
auth = tweepy.OAuth2BearerHandler(bearer_token)
api = tweepy.API(auth)


In [ ]:
#retriving a tweet from home page
# public_tweets = api.home_timeline()
# public_tweets[0].text

### Twitter API rate limits

In [ ]:
### checking rate limit - friends list
limit = api.rate_limit_status()
limit['resources']['friends']['/friends/list']['remaining']
limit['resources']['friends']['/friends/list']

#### Calculating remaining time (in mins) before api limits reset
I find it helpful to check how many more minutes I have to wait before trying again

In [ ]:
import datetime as dt
given_date =dt.datetime.fromtimestamp(
        int(limit['resources']['friends']['/friends/list']['reset'])
    ).strftime('%Y-%m-%d %H:%M:%S')
difference =  dt.datetime.strptime(given_date, "%Y-%m-%d %H:%M:%S")-dt.datetime.today()
min=difference.seconds/60
print ("Minute(s) remaining: ",min)

### Key function(s) of tweepy wrapper
These fields are are stored in Neo4j (as properties) against a Person node.

In [ ]:
# getting user details of a Person aka Narendra Modi
# any other Person details can be fetched as a starting point 
user_input = input('Enter some twitter handle:')
suser = api.get_user(screen_name=user_input) # this returns a User model
print (suser.screen_name)
print (suser.followers_count)
print (suser.friends_count)
print(suser.location)
print(suser.created_at)

In [ ]:
# get Narendra Modi's friends i.e. whome NM is following
friends = api.get_friends(screen_name=user_input)

In [ ]:
# defining a driver
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "admin"))
# initiating a server
session = driver.session()

#adding a unique constraint; this ensures that same Person is not added twice
try:
    #deleting ALL existing records
    #session.run("MATCH (n) DETACH DELETE n")
    session.run("CREATE CONSTRAINT ON (a:Person) ASSERT a.screen_name IS UNIQUE")
except Exception as e:
    print(e)

session.close()

In [ ]:
# defining a driver
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "admin"))
session = driver.session()

#adding user details
label =suser.screen_name
session.run("CREATE (label:Person {screen_name: $screen_name, name: $name, followers_count: $followers_count, friends_count:$friends_count,location:$location})",
              {"name": suser.name, "screen_name": suser.screen_name, "followers_count":suser.followers_count,"friends_count": suser.friends_count,"location":suser.location }
               )
session.close()

In [ ]:
# defining a driver
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "admin"))
session = driver.session()

i =0


#adding all persons - whom the target user is following
for user in friends:
    #checking if the user already exists
    output = session.run("MATCH (a:Person) WHERE a.screen_name =$check_name return a.screen_name",
                             {"check_name":user.screen_name})
    exists =''
    for exists in output:
            True
    label =user.screen_name
    #adding the user if doesnt exist
    if exists=='':
        session.run("CREATE (label:Person {screen_name: $screen_name, name: $name, followers_count: $followers_count, friends_count:$friends_count,location:$location})",
              {"name": user.name, "screen_name": user.screen_name, "followers_count":user.followers_count,"friends_count": user.friends_count,"location":user.location }
               )
    else:
            print('Person already exists: ',user.screen_name)
            
    session.run("MATCH (a:Person),(b:Person) WHERE a.screen_name = $a_screen_name AND b.screen_name = $b_screen_name CREATE (a)-[r:FOLLOWING]->(b)",
               {"a_screen_name":suser.screen_name, "b_screen_name":user.screen_name })
    
    #temporary; ending the loop after 6 runs - dont exhaust twitter api limits
    i=i+1
    if i==6:
        break
    
    #finding friends of Modi's friends
    print ('sub_friends: '+ user.screen_name) #debugging why rate limits are getting exhausted
    sub_friends = api.get_friends(screen_name=user.screen_name)
    #adding the friends of friends of Modi
    for sub_user in sub_friends:
        #checking if the sub_user already exists
        output = session.run("MATCH (a:Person) WHERE a.screen_name =$check_name return a.screen_name",
                             {"check_name":sub_user.screen_name})
        exists =''
        for exists in output:
            True
        label =sub_user.screen_name
        #adding the user if doesnt exist
        if exists=='':
            
            session.run("CREATE (label:Person {screen_name: $screen_name, name: $name, followers_count: $followers_count, friends_count:$friends_count,location:$location})",
              {"name": sub_user.name, "screen_name": sub_user.screen_name, "followers_count":sub_user.followers_count,"friends_count": sub_user.friends_count,"location":sub_user.location }
               )
        else:
            print('Person already exists: ',sub_user.screen_name)
        session.run("MATCH (a:Person),(b:Person) WHERE a.screen_name = $a_screen_name AND b.screen_name = $b_screen_name CREATE (a)-[r:FOLLOWING]->(b)",
               {"a_screen_name":user.screen_name, "b_screen_name":sub_user.screen_name })
        

session.close()